# Práctica 1 Solución :Web Scraping con Selenium

## Tratamiento de datos masivos - Facultad de Informática - Rafael Caballero

Nombre: 

**Inicialización:**

* Descargar el fichero 'chromedriver.exe'. Alojarlo en una carpeta local. En el resto de la práctica se asume que se situa en la carpeta' c:/hlocal/tdm
* Ejecutar el siguiente código y comprobar que se abre una sesión de Chrome

In [8]:
# https://sites.google.com/a/chromium.org/chromedriver/downloads
import os
from selenium import webdriver  # si da error, desde anaconda prompt hacer pip install --user  selenium
chromedriver = "c:/hlocal/tdm/chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(executable_path=chromedriver)

1.- Escribir código para abrir en Chrome la página: http://clearskycalculator.com/pyranometer.htm

In [9]:
url = 'http://clearskycalculator.com/pyranometer.htm'
driver.get(url)

2.- La página nos permite calcular la radiación esperada para una cierta longitud y latitud. 
Escribir el código que 

2.1 Escribe en el formulario los valores 41.4 (latitud) y 355.5 (longitud) y 345 (longitud tz). 
Notas: 
* Para conocer los datos que identifican a un objeto colocarse sobre él y pulsar inspeccionar. La primera vez hay que hacerlo 2 veces.
* Para escribir texto en una caja input se utiliza el método send_keys(s), con s el string que se desea escribir. 
Antes de llamar a este método conviene llamar al método clear del input para borrar el valor antiguo.
* Utilizar el id para identificar objetos cuando sea posible. Recordar que el nombre de identificadores tiene en cuenta mayúscula y minúscula

In [19]:
lat = driver.find_element_by_id("p1G6")
lat.clear()
lat.send_keys("41.4")
long = driver.find_element_by_id("p1G8")
long.clear()
long.send_keys("355.5")
longz = driver.find_element_by_id("p1G10")
longz.clear()
longz.send_keys("345")

2.2 Pulsar el botón "RECALCULATE MODEL". 
Nota:  para obtener un elemento por un valor de un atributo se escribe 
``` [Python]
elemento = driver.findElement_by_xpath("//element[@attribute='púlsame']"))
```

In [41]:
boton = driver.find_element_by_xpath('//input[@value="RECALCULATE MODEL"]')
b.click()


2.3 Obtener y mostrar por pantalla el valor de radiación solar de la etiqueta a la derecha texto *Model Estimated Shortwave*
Nota: Para obtener el texto asociado a un input fijarse en esta pregunta:
https://stackoverflow.com/questions/25580569/get-value-of-an-input-box-using-selenium-python

In [52]:
rad = driver.find_element_by_id("p1L6")
print(rad.get_attribute('value'))

976


3 En Python las funciones nos permiten agrupar código que vamos a utilizar varias veces. Por ejemplo:


In [53]:
def mayor(x,y):
    z = max(x,y)
    return z

# llamamos a la función
mayor(3,4)

4

Reunir el código anterior en una función que reciba como parámetros los valores lat, lon, lonz y devuelva la radiación esperada:
    

In [56]:
def radiacion(latitud,longitud,longitudz): #### esta linea no se puede cambiar
    ### código de 2.1, adaptado para que send_keys utilice los parámetros latitud, longitud longitudz
    lat = driver.find_element_by_id("p1G6")
    lat.clear()
    lat.send_keys(latitud)
    long = driver.find_element_by_id("p1G8")
    long.clear()
    long.send_keys(longitud)
    longz = driver.find_element_by_id("p1G10")
    longz.clear()
    longz.send_keys(longitudz)
    ### código de 2.2
    boton = driver.find_element_by_xpath('//input[@value="RECALCULATE MODEL"]')
    b.click()
    ###código de 2.3
    rad = driver.find_element_by_id("p1L6")
    # print(rad.get_attribute('value'))
    return  rad.get_attribute('value')  ## añadir tras return la expresión o variable que contiene el valor de raciación

Para probarlo podemos ejecutar:

In [57]:
## pruebas
print(radiacion("41.7","32.3","0"))

855


4  Ahora queremos repetir el trabajo pero leyendo los datos de latitu, longitud y longitudz del fichero *datos.csv*. En el fichero los valores están con la forma *lat,lon,lonz*, y no hay cabecera. Se pide:

4.1 Leer el fichero, que debemos alojar en la carpeta c:/hlocal/tdm. Leerlo como csv (ver ejercicio 1 de la práctica 2)

4.2 Para cada linea del fichero llamaremos a la función *radiación* e iremos mostrando tanto los valores leídos, como el valor devuelto por la función.

La salida será similar a:

```
41.4 71.2 74 --> 995
38.3 68.2 74 --> 1011
77.3 258.2 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906
```


In [65]:
### Solución 4.1, 4.2
import csv
path = 'c:/hlocal/tdm/datos.csv'
with open(path) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        print(row[0],row[1],row[2],'-->',radiacion(row[0],row[1],row[2]))

41.4 71.2 74 --> 995
38.3 68.2 74 --> 1011
77.3 258.2 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906


5 Ahora consideramos el ficheros *datosmissing.csv*  
Queremos hacer lo mismo que en el ejercicio 4, pero la diferencia es que, en este caso faltan algunos valores para la segunda columna, la longitud.

Estos valores 'missing' son muy comunes en ciencia de datos y hay varias formas de tratarlos, que vamos a probar a continuación:

5.1 Lo primero es conocer cuántos datos de este tipo hay. Escribir un código que lea el fichero *datosmissing* 
y cuente el número de filas cuya longitud tiene longitud 0. También debe indicar el % de filas con esta característica.
La salida será:
```
Total de filas missing:  8  lo que representa el  22.857142857142858  por ciento del total
```

In [72]:
import csv
path = 'c:/hlocal/tdm/datosmissing.csv'
### solución 5.1
n=0
m=0
with open(path) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        if row[1]=='':
           m = m+1
        n = n+1
print('Total de filas missing: ', m, ' lo que representa el ',m*100.0/n,' por ciento del total')
            

Total de filas missing:  8  lo que representa el  22.857142857142858  por ciento del total


5.2 La primera forma es simplemente ignorar las filas que existen estos huecos. Escribir un código similar al del ejercicio 4,
pero que cuando el valor de la longitud es vacío (len(x) == 0, o x=='', lo que se prefiera), no llame a la función radiacion sino que escriba la línea missing.

La salida esperada será de la forma:
```
41.4 71.2 74 --> 995
*missing*
77.3 258.2 260 --> 576
*missing*
52.3 6 7 --> 906
...
```

In [67]:
import csv
path = 'c:/hlocal/tdm/datosmissing.csv'
### solución 5.2
with open(path) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        if row[1]=='':
            print('*missing*')
        else: 
            print(row[0],row[1],row[2],'-->',radiacion(row[0],row[1],row[2]))

41.4 71.2 74 --> 995
*missing*
77.3 258.2 260 --> 576
*missing*
52.3 6 7 --> 906
41.4 71.2 74 --> 995
38.3 68.2 74 --> 1011
*missing*
89.3 4 0 --> 373
52.3 6 7 --> 906
41.4 71.2 74 --> 995
38.3 68.2 74 --> 1011
77.3 258.2 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906
41.4 71.2 74 --> 995
38.3 68.2 74 --> 1011
77.3 258.2 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906
41.4 71.2 74 --> 995
38.3 68.2 74 --> 1011
77.3 258.2 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906
41.4 71.2 74 --> 995
38.3 68.2 74 --> 1011
77.3 258.2 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906
41.4 71.2 74 --> 995
38.3 68.2 74 --> 1011
77.3 258.2 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906


5.3 La segunda posibilidad es *rellenar* el valor. En ocasiones se hará con la media de otros valores. En este caso lo que haremos será tomar como valor de longitud el valor longitudz cuando longitud no exista. Salida esperada:

41.4 71.2 74 --> 995

38.3 **74** 74 --> 1012

77.3 258.2 260 --> 576

89.3 **0** 0 --> 373

52.3 6 7 --> 906

...

In [74]:
import csv
path = 'c:/hlocal/tdm/datosmissing.csv'
### solución 5.3
with open(path) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        if row[1]=='':
            row[1] = row[2]
        print(row[0],row[1],row[2],'-->',radiacion(row[0],row[1],row[2]))

41.4 71.2 74 --> 995
38.3 74 74 --> 1012
77.3 258.2 260 --> 576
89.3 0 0 --> 373
52.3 6 7 --> 906
41.4 71.2 74 --> 995
38.3 68.2 74 --> 1011
77.3 260 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906
41.4 71.2 74 --> 995
38.3 68.2 74 --> 1011
77.3 258.2 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906
41.4 71.2 74 --> 995
38.3 68.2 74 --> 1011
77.3 258.2 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906
41.4 74 74 --> 994
38.3 74 74 --> 1012
77.3 260 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906
41.4 74 74 --> 994
38.3 68.2 74 --> 1011
77.3 258.2 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906
41.4 71.2 74 --> 995
38.3 74 74 --> 1012
77.3 258.2 260 --> 576
89.3 4 0 --> 373
52.3 6 7 --> 906
